### Cuaderno de exploración del analisis estadistico
Este cuaderno desarrolla la exploración de las fuentes con datos, que anteriormente fueron separadas en archivos independientes por cada estación.

Empezamos leyendo las fuentes independientes generadas por el paso de Lectura de Fuentes y separación por archivos.

In [2]:
import shutil
import os
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

def annotate_boxplot_outliers(data, x_col, y_col, label_col, ax):
    cont = 0
    for i, row in data.iterrows():
        x = row[x_col]
        y = row[y_col]
        label = row[label_col]

        # Calcular los percentiles para detectar outliers
        Q1 = data[data[x_col] == x][y_col].quantile(0.25)
        Q3 = data[data[x_col] == x][y_col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        if y < lower_bound or y > upper_bound:
            cont += 1
            orient = (10, 0) if cont % 2 == 0 else (-30, 0)
            ax.annotate(label, (x, y), textcoords="offset points", xytext=orient, ha='left', va='center')

In [3]:
lista_zonas = ['MAGDALENA_MEDIO']
path = '../datos/separados_por_estacion/'

In [11]:
for zona in lista_zonas:
    print('Zona:',zona)
    path_zone = path+zona

    for file in os.listdir(path_zone):
        print('Archivo:',file)
        col = file.split('.')[0]

        shutil.rmtree(f'../salidas/{zona}/{col}/graficos_descriptivos/', ignore_errors=True)
        path_outputs = f'../salidas/{zona}/{col}/graficos_descriptivos/'

        file_path = path_zone + '/' + file
        df = pd.read_csv(file_path, sep=',')

        # Descripcion de los datos
        # print('Estadisticas:',df.describe()) # Revisar que valores guardar a nivel de estadisticas
        # print('Nulos:',df.isnull().sum())

        # Generamos bloxplot por mes
        plt.figure(figsize=(14, 6))
        ax = sns.boxplot(y=col, x='Mes', data=df, whis=1.5)
        annotate_boxplot_outliers(df, 'Mes', col, 'Año', ax)
        plt.title(f'Boxplot de precipitación de la estación {col} por Mes')
        plt.xlabel("Estacion: "+ col)
        plt.ylabel('Precipitación (mm)')
        os.makedirs(path_outputs, exist_ok=True)
        plt.legend()
        plt.savefig(f'{path_outputs}{col}.png')
        plt.close()

        # Generamos diagramas de barras por cada año
        os.makedirs(path_outputs + "graficos_barras/", exist_ok=True)
        for year in df['Año'].unique():
            plt.figure(figsize=(14, 6))
            sns.barplot(x='Mes', y=col, data=df[df['Año'] == year], palette='viridis')
            plt.title(f'Precipitación por mes en la estación {col}, para el año {year}')
            plt.xlabel(year)
            plt.ylabel('Precipitación (mm)')
            plt.savefig(f'{path_outputs}graficos_barras/{col}_{year}.png')
            plt.close()
        
        os.makedirs(path_outputs + "histograma/", exist_ok=True)
        for month in df['Mes'].unique():
            plt.figure(figsize=(14, 6))
            plt.hist(df[df['Mes'] == month][col], edgecolor='k', alpha=0.7)
            plt.title(f'Histograma de precipitaciones para: {month}')
            plt.xlabel('Precipitaciones')
            plt.ylabel('Frecuencia')
            plt.grid(True)
            plt.savefig(f'{path_outputs}histograma/{col}_{month}.png')
            plt.close()

Zona: MAGDALENA_MEDIO
Archivo: 23010020.csv
Archivo: 23010080.csv
Archivo: 23020080.csv
Archivo: 23020090.csv
Archivo: 23020100.csv
Archivo: 23025040.csv
Archivo: 23040070.csv
Archivo: 23050080.csv
Archivo: 23050100.csv
Archivo: 23050230.csv
Archivo: 23050250.csv
Archivo: 23055040.csv
Archivo: 23055070.csv
Archivo: 23060110.csv
Archivo: 23060140.csv
Archivo: 23060150.csv
Archivo: 23060160.csv
Archivo: 23060170.csv
Archivo: 23060180.csv
Archivo: 23060190.csv
Archivo: 23060200.csv
Archivo: 23060260.csv
Archivo: 23060290.csv
Archivo: 23060370.csv
Archivo: 23065120.csv
Archivo: 23070010.csv
Archivo: 23070020.csv
Archivo: 23080390.csv
Archivo: 23080640.csv
Archivo: 23080650.csv
Archivo: 23080720.csv
Archivo: 23080740.csv
Archivo: 23080750.csv
Archivo: 23080760.csv
Archivo: 23080810.csv
Archivo: 23080820.csv
Archivo: 23080920.csv
Archivo: 23080940.csv
Archivo: 23085030.csv
Archivo: 23085080.csv
Archivo: 23085110.csv
Archivo: 23085140.csv
Archivo: 23085160.csv
Archivo: 23085220.csv
Archivo: 2